## Importing data (Alex updated: 4/30/19)

In [1]:
import numpy as np

# Image processing tools
import skimage
import skimage.filters

import pandas as pd
import scipy.optimize
import scipy.stats as st
import numba
import itertools

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from numpy.linalg import inv

from tqdm import tqdm, trange
from scipy import special, optimize
from scipy import integrate

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
#plotly.tools.set_credentials_file(username='AYChoi', api_key='ZacDa7fKo8hfiELPfs57')
plotly.tools.set_credentials_file(username='AlexanderYChoi', api_key='VyLt05wzc89iXwSC82FO')

Load the e-ph matrix elements data. The two numbers reported at the end should be the same. If they are not, there are duplicate e-ph elements. It's VERY IMPORTANT to note that the g elements here are actually |g|^2 which is why they are real numbers.

In [2]:
data = pd.read_csv('gaas.eph_matrix', sep='\t',header= None,skiprows=(0,1))
data.columns = ['0']
data_array = data['0'].values
new_array = np.zeros((len(data_array),7))
for i1 in trange(len(data_array)):
    new_array[i1,:] = data_array[i1].split()
    
g_df = pd.DataFrame(data=new_array,columns = ['k_inds','q_inds','k+q_inds','m_band','n_band','im_mode','g_element'])
g_df[['k_inds','q_inds','k+q_inds','m_band','n_band','im_mode']] = g_df[['k_inds','q_inds','k+q_inds','m_band','n_band','im_mode']].apply(pd.to_numeric,downcast = 'integer')
len(g_df[['k_inds','q_inds','k+q_inds','m_band','n_band','im_mode','g_element']]),len(g_df[['k_inds','q_inds','k+q_inds','m_band','n_band','im_mode','g_element']].drop_duplicates())

100%|████████████████████████████████████████████████████████████████████| 7854608/7854608 [00:34<00:00, 227060.91it/s]


(7854608, 7854608)

Now load the k-point indices, q-point indices, k-point energies, phonon energies into dataframes.

In [3]:
kpts = pd.read_csv('gaas.kpts', sep='\t',header= None)
kpts.columns = ['0']
kpts_array = kpts['0'].values
new_kpt_array = np.zeros((len(kpts_array),4))
for i1 in trange(len(kpts_array)):
    new_kpt_array[i1,:] = kpts_array[i1].split()
    
kpts_df = pd.DataFrame(data=new_kpt_array,columns = ['k_inds','b1','b2','b3'])
kpts_df[['k_inds']] = kpts_df[['k_inds']].apply(pd.to_numeric,downcast = 'integer')
kpts_df.head()

100%|██████████████████████████████████████████████████████████████████████████| 2213/2213 [00:00<00:00, 317029.67it/s]


,k_inds,b1,b2,b3
0,1,0.0,0.0,0.00
1,2,0.0,0.0,0.01
2,3,0.0,0.0,0.02
3,4,0.0,0.0,0.03
4,5,0.0,0.0,0.04


In [4]:
enk = pd.read_csv('gaas.enk', sep='\t',header= None)
enk.columns = ['0']
enk_array = enk['0'].values
new_enk_array = np.zeros((len(enk_array),3))
for i1 in trange(len(enk_array)):
    new_enk_array[i1,:] = enk_array[i1].split()
    
enk_df = pd.DataFrame(data=new_enk_array,columns = ['k_inds','band_inds','energy [Ryd]'])
enk_df[['k_inds','band_inds']] = enk_df[['k_inds','band_inds']].apply(pd.to_numeric,downcast = 'integer')
enk_df.head()

100%|██████████████████████████████████████████████████████████████████████████| 2213/2213 [00:00<00:00, 316964.72it/s]


,k_inds,band_inds,energy [Ryd]
0,1,1,0.445788
1,2,1,0.447866
2,3,1,0.453308
3,4,1,0.460652
4,5,1,0.468793


In [5]:
enq = pd.read_csv('gaas.enq', sep='\t',header= None)
enq.columns = ['0']
enq_array = enq['0'].values
new_enq_array = np.zeros((len(enq_array),3))
for i1 in trange(len(enq_array)):
    new_enq_array[i1,:] = enq_array[i1].split()
    
enq_df = pd.DataFrame(data=new_enq_array,columns = ['q_inds','im_mode','energy [Ryd]'])
enq_df[['q_inds','im_mode']] = enq_df[['q_inds','im_mode']].apply(pd.to_numeric,downcast = 'integer')
enq_df.head()

100%|██████████████████████████████████████████████████████████████████████| 126480/126480 [00:00<00:00, 309312.25it/s]


,q_inds,im_mode,energy [Ryd]
0,1,1,0.000020
1,1,2,0.000026
2,1,3,0.000054
3,1,4,0.002432
4,1,5,0.002432


In [6]:
qpts = pd.read_csv('gaas.qpts', sep='\t',header= None)
qpts.columns = ['0']
qpts_array = qpts['0'].values
new_qpt_array = np.zeros((len(qpts_array),4))

for i1 in trange(len(qpts_array)):
    new_qpt_array[i1,:] = qpts_array[i1].split()
    
qpts_df = pd.DataFrame(data=new_qpt_array,columns = ['q_inds','b1','b2','b3'])
qpts_df[['q_inds']] = qpts_df[['q_inds']].apply(pd.to_numeric,downcast = 'integer')
qpts_df.head()

100%|████████████████████████████████████████████████████████████████████████| 21080/21080 [00:00<00:00, 325178.11it/s]


,q_inds,b1,b2,b3
0,1,0.0,0.0,0.01
1,2,0.0,0.0,0.02
2,3,0.0,0.0,0.03
3,4,0.0,0.0,0.04
4,5,0.0,0.0,0.05


## Data Processing (Alex Updated: 4/30)

In [8]:
def cartesian_k_points(kpts_df):
    """
    Given a dataframe containing indexed k-points in terms of the crystal lattice vector, return the dataframe with cartesian k coordinates.     
    Parameters:
    -----------
    kpts_df : pandas dataframe containing:
        
        k_inds : vector_like, shape (n,1)
        Index of k point
        
        kx : vector_like, shape (n,1)
        x-coordinate in momentum space [1/A]    
        
        ky : vector_like, shape (n,1)
        y-coordinate in momentum space [1/A]  
        
        kz : vector_like, shape (n,1)
        z-coordinate in momentum space [1/A]
        
    For FCC lattice, use the momentum space primitive vectors as per:
    http://lampx.tugraz.at/~hadley/ss1/bzones/fcc.php
    
    b1 = 2 pi/a (kx - ky + kz)
    b2 = 2 pi/a (kx + ky - kz)
    b3 = 2 pi/a (-kx + ky + kz)
    
    Returns:
    --------
    cart_kpts_df : pandas dataframe containing:
    
        k_inds : vector_like, shape (n,1)
        Index of k point
        
        kx : vector_like, shape (n,1)
        x-coordinate in Cartesian momentum space [1/m]    
        
        ky : vector_like, shape (n,1)
        y-coordinate in Cartesian momentum space [1/m]  
        
        kz : vector_like, shape (n,1)
        z-coordinate in Cartesian momentum space [1/m]  
    """
    
    # Need a lattice constant for GaAs. This is obviously somewhat sensitive to temperature.
    a = 5.65325*10**(-10) #[m]
    
    cartesian_df = pd.DataFrame(columns = ['k_inds','kx [1/m]','ky [1/m]','kz [1/m]'])
    
    cartesian_df['kx [1/m]'] = (kpts_df['b1'].values + kpts_df['b2'].values)/2*(a/np.pi)/2
    cartesian_df['ky [1/m]'] = (kpts_df['b2'].values + kpts_df['b3'].values)/2*(a/np.pi)/2
    cartesian_df['kz [1/m]'] = (kpts_df['b1'].values + kpts_df['b3'].values)/2*(a/np.pi)/2
    cartesian_df['k_inds'] = kpts_df['k_inds'].values
    return cartesian_df

In [14]:
def cartesian_q_points(qpts_df):
    """
    Given a dataframe containing indexed q-points in terms of the crystal lattice vector, return the dataframe with cartesian q coordinates.     
    Parameters:
    -----------
    qpts_df : pandas dataframe containing:
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        kx : vector_like, shape (n,1)
        x-coordinate in momentum space [1/A]    
        
        ky : vector_like, shape (n,1)
        y-coordinate in momentum space [1/A]  
        
        kz : vector_like, shape (n,1)
        z-coordinate in momentum space [1/A]
        
    For FCC lattice, use the momentum space primitive vectors as per:
    http://lampx.tugraz.at/~hadley/ss1/bzones/fcc.php
    
    b1 = 2 pi/a (kx - ky + kz)
    b2 = 2 pi/a (kx + ky - kz)
    b3 = 2 pi/a (-kx + ky + kz)
    
    Returns:
    --------
    cart_kpts_df : pandas dataframe containing:
    
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        kx : vector_like, shape (n,1)
        x-coordinate in Cartesian momentum space [1/m]    
        
        ky : vector_like, shape (n,1)
        y-coordinate in Cartesian momentum space [1/m]  
        
        kz : vector_like, shape (n,1)
        z-coordinate in Cartesian momentum space [1/m]  
    """
    
    # Need a lattice constant for GaAs. This is obviously somewhat sensitive to temperature.
    a = 5.65325*10**(-10) #[m]
    
    cartesian_df = pd.DataFrame(columns = ['q_inds','kx [1/m]','ky [1/m]','kz [1/m]'])
    
    cartesian_df['kx [1/m]'] = (qpts_df['b1'].values + qpts_df['b2'].values)/2*(a/np.pi)/2
    cartesian_df['ky [1/m]'] = (qpts_df['b2'].values + qpts_df['b3'].values)/2*(a/np.pi)/2
    cartesian_df['kz [1/m]'] = (qpts_df['b1'].values + qpts_df['b3'].values)/2*(a/np.pi)/2
    cartesian_df['q_inds'] = qpts_df['q_inds'].values
    return cartesian_df

In [15]:
cart_kpts_df = cartesian_k_points(kpts_df)
cart_qpts_df = cartesian_q_points(qpts_df)

In [16]:
a = 5.65325*10**(-10) #[m]

In [20]:
trace1 = go.Scatter3d(
    x=cart_kpts_df['kx [1/m]'].values*(2*np.pi/(a)),
    y=cart_kpts_df['ky [1/m]'].values*(2*np.pi/(a)),
    z=cart_kpts_df['kz [1/m]'].values*(2*np.pi/(a)),
    mode='markers',
    marker=dict(
        size=2,
        color=enk_df['energy [Ryd]'],
        colorscale='Rainbow',
        showscale=True,
        opacity=1
    )
)

data = [trace1]
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title='kx',titlefont = dict(family='Oswald, monospace',size=18)),
                    yaxis = dict(
                        title='ky',titlefont = dict(family='Oswald, monospace',size=18)),
                    zaxis = dict(
                        title='kz',titlefont = dict(family='Oswald, monospace',size=18)),))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~AlexanderYChoi/0 or inside your plot.ly account where it is named 'simple-3d-scatter'


In [19]:
trace1 = go.Scatter3d(
    x=cart_qpts_df['kx [1/m]'].values*(2*np.pi/(a)),
    y=cart_qpts_df['ky [1/m]'].values*(2*np.pi/(a)),
    z=cart_qpts_df['kz [1/m]'].values*(2*np.pi/(a)),
    mode='markers',
    marker=dict(
        size=2,
        opacity=1
    )
)

data = [trace1]
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title='kx',titlefont = dict(family='Oswald, monospace',size=18)),
                    yaxis = dict(
                        title='ky',titlefont = dict(family='Oswald, monospace',size=18)),
                    zaxis = dict(
                        title='kz',titlefont = dict(family='Oswald, monospace',size=18)),))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~AlexanderYChoi/0 or inside your plot.ly account where it is named 'simple-3d-scatter'


In [21]:
def fermi_distribution(g_df,mu,T):
    """
    This function takes a list of k-point indices and returns the Fermi-distributions and energies associated with each k-point on that list. The Fermi distributions are calculated with respect to a particular chemical potential.      
    Parameters:
    -----------
    
    g_df : pandas dataframe containing:
    
        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
        
        k_energy : vector_like, shape (n,1)
        Energy of the pre collision state
        
        k+q_energy : vector_like, shape (n,1)
        Energy of the post collision state
        
        
    mu : scalar
    Chemical potential of electronic states [eV]
    
    T : scalar
    Lattice temperature in Kelvin
    
    Returns:
    --------
    
    g_df : pandas dataframe containing:

        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
        
        k_fermi_dist : vector_like, shape (n,1)
        Fermi distribution of pre collision state
        
        k+q_fermi_dist : vector_like, shape (n,1)
        Fermi distribution of post collision state
        
        k_energy : vector_like, shape (n,1)
        Energy of the pre collision state
        
        k+q_energy : vector_like, shape (n,1)
        Energy of the post collision state
         
    """
    # Physical constants    
    e = 1.602*10**(-19) # fundamental electronic charge [C]
    kb = 1.38064852*10**(-23); # Boltzmann constant in SI [m^2 kg s^-2 K^-1]


    g_df['k_FD'] = (np.exp((g_df['k_en [eV]'].values*e - mu*e)/(kb*T)) + 1)**(-1)
    g_df['k+q_FD'] = (np.exp((g_df['k+q_en [eV]'].values*e - mu*e)/(kb*T)) + 1)**(-1)

    return g_df

In [22]:
def bose_distribution(g_df,T):
    """
    This function takes a list of q-point indices and returns the Bose-Einstein distributions associated with each q-point on that list.    
    Parameters:
    -----------
    
    g_df : pandas dataframe containing:
    
        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
        
        k_energy : vector_like, shape (n,1)
        Energy of the pre collision state
        
        k+q_energy : vector_like, shape (n,1)
        Energy of the post collision state
        
        
    mu : scalar
    Chemical potential of electronic states [eV]
    
    T : scalar
    Lattice temperature in Kelvin
    
    Returns:
    --------
    
    g_df : pandas dataframe containing:

        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
        
        BE : vector_like, shape (n,1)
        Bose-einstein distribution
         
    """
    # Physical constants    
    e = 1.602*10**(-19) # fundamental electronic charge [C]
    kb = 1.38064852*10**(-23); # Boltzmann constant in SI [m^2 kg s^-2 K^-1]


    g_df['BE'] = (np.exp((g_df['q_en [eV]'].values*e)/(kb*T)) - 1)**(-1)
    return g_df

In [23]:
def fermionic_processing(g_df,cart_kpts_df,enk_df,mu,T):
    """
    This function takes a list of k-point indices and returns the Fermi-distributions and energies associated with each k-point on that list. The Fermi distributions are calculated with respect to a particular chemical potential.      
    Parameters:
    -----------
    
    g_df : pandas dataframe containing:
    
        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
               
    cart_kpts_df : pandas dataframe containing:
    
        k_inds : vector_like, shape (n,1)
        Index of k point
        
        kx : vector_like, shape (n,1)
        x-coordinate in Cartesian momentum space [1/m]    
        
        ky : vector_like, shape (n,1)
        y-coordinate in Cartesian momentum space [1/m]  
        
        kz : vector_like, shape (n,1)
        z-coordinate in Cartesian momentum space [1/m]
        
    enk_df : pandas dataframe containing

        k_inds : vector_like, shape (n,1)
        Index of k point
        
        band_inds : vector_like, shape (n,1)
        Band index
        
        energy [Ryd] : vector_like, shape (n,1)
        Energy associated with k point in Rydberg units
        
        
    mu : scalar
    Chemical potential of electronic states [eV]
    
    T : scalar
    Lattice temperature in Kelvin
    
    Returns:
    --------
    
    g_df : pandas dataframe containing:

        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
        
        k_fermi_dist : vector_like, shape (n,1)
        Fermi distribution of pre collision state
        
        k+q_fermi_dist : vector_like, shape (n,1)
        Fermi distribution of post collision state
        
        k_energy : vector_like, shape (n,1)
        Energy of the pre collision state
        
        k+q_energy : vector_like, shape (n,1)
        Energy of the post collision state
         
    """
    
    # Physical constants
    e = 1.602*10**(-19) # fundamental electronic charge [C]
    kb = 1.38064852*10**(-23); # Boltzmann constant in SI [m^2 kg s^-2 K^-1]
    
    index_vector = cart_kpts_df['k_inds'].values    
    g_df['k_en [eV]'] = np.zeros(len(g_df))
    g_df['k+q_en [eV]'] = np.zeros(len(g_df))
    g_df['collision_state'] = np.zeros(len(g_df))
    
    for i1 in trange(len(cart_kpts_df)):
        index = index_vector[i1]
        
        g_slice_k = g_df['k_inds'] == index        
        g_slice_kq = g_df['k+q_inds'] == index
        
        k_slice = cart_kpts_df['k_inds'] == index
        enk_slice = enk_df['k_inds'] == index
        
        g_df.loc[g_slice_k,'k_en [eV]'] = enk_df.loc[enk_slice,'energy [Ryd]'].values*13.6056980659 
        g_df.loc[g_slice_kq,'k+q_en [eV]'] = enk_df.loc[enk_slice,'energy [Ryd]'].values*13.6056980659
        
    abs_inds = g_df['k_en [eV]'] < g_df['k+q_en [eV]'] #absorbed indices
    ems_inds = g_df['k_en [eV]'] > g_df['k+q_en [eV]'] #emission indices
    
    g_df.loc[abs_inds,'collision_state'] = 1
    g_df.loc[ems_inds,'collision_state'] = -1
    
    g_df = fermi_distribution(g_df,mu, T)
    
    return g_df

In [24]:
g_df = fermionic_processing(g_df,cart_kpts_df,enk_df,5.780,300)

100%|██████████████████████████████████████████████████████████████████████████████| 2213/2213 [08:25<00:00,  4.49it/s]


In [25]:
def bosonic_processing(g_df,cart_qpts_df,enq_df,T):
    """
    This function takes a list of k-point indices and returns the Fermi-distributions and energies associated with each k-point on that list. The Fermi distributions are calculated with respect to a particular chemical potential.      
    Parameters:
    -----------
    
    g_df : pandas dataframe containing:
    
        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
               
    cart_qpts_df : pandas dataframe containing:
    
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        kx : vector_like, shape (n,1)
        x-coordinate in Cartesian momentum space [1/m]    
        
        ky : vector_like, shape (n,1)
        y-coordinate in Cartesian momentum space [1/m]  
        
        kz : vector_like, shape (n,1)
        z-coordinate in Cartesian momentum space [1/m]
        
    enq_df : pandas dataframe containing

        k_inds : vector_like, shape (n,1)
        Index of k point
        
        im_mode : vector_like, shape (n,1)
        Phonon polarization index
        
        energy [Ryd] : vector_like, shape (n,1)
        Energy associated with k point in Rydberg units
        
            
    T : scalar
    Lattice temperature in Kelvin
    
    Returns:
    --------
    
    g_df : pandas dataframe containing:

        k_inds : vector_like, shape (n,1)
        Index of k point (pre-collision)
        
        q_inds : vector_like, shape (n,1)
        Index of q point
        
        k+q_inds : vector_like, shape (n,1)
        Index of k point (post-collision)
        
        m_band : vector_like, shape (n,1)
        Band index of post-collision state
        
        n_band : vector_like, shape (n,1)
        Band index of pre-collision state
        
        im_mode : vector_like, shape (n,1)
        Polarization of phonon mode
        
        g_element : vector_like, shape (n,1)
        E-ph matrix element
        
        k_fermi_dist : vector_like, shape (n,1)
        Fermi distribution of pre collision state
        
        k+q_fermi_dist : vector_like, shape (n,1)
        Fermi distribution of post collision state
        
        k_energy : vector_like, shape (n,1)
        Energy of the pre collision state
        
        k+q_energy : vector_like, shape (n,1)
        Energy of the post collision state
         
    """
    
    # Physical constants
    e = 1.602*10**(-19) # fundamental electronic charge [C]
    kb = 1.38064852*10**(-23); # Boltzmann constant in SI [m^2 kg s^-2 K^-1]
    
    index_vector = cart_qpts_df['q_inds'].values
    polar_vector = np.unique(g_df['im_mode'].values)
    
    g_df['q_en [eV]'] = np.ones(len(g_df))
    
    for i1 in trange(len(cart_qpts_df)):
    #for i1 in trange(10):

        index = index_vector[i1]
        index_slice = enq_df['q_inds'].values == index
        
        g_slice_q = g_df['q_inds'] == index        
                
        for i2 in range(len(polar_vector)):
            polar_index = polar_vector[i2]
            polar_slice = enq_df['im_mode'].values == polar_index
            hybrid_slice = np.multiply(index_slice,polar_slice)
            g_slice_im = g_df['im_mode'] == polar_index
            hybrid_g_slice = np.multiply(g_slice_q,g_slice_im)

            g_df.loc[hybrid_g_slice,'q_en [eV]'] = enq_df.loc[hybrid_slice,'energy [Ryd]'].values*13.6056980659
            
            #print(enq_df.loc[hybrid_slice,'energy [Ryd]'].values)
        
    
    g_df = bose_distribution(g_df, T)
    
    return g_df

In [ ]:
g_df = bosonic_processing(g_df,cart_qpts_df,enq_df,300)

  2%|█▊                                                                          | 504/21080 [06:08<4:32:18,  1.26it/s]

In [ ]:
g_df_ems = g_df.loc[g_df['collision_state'] > 0]
g_df_abs = g_df.loc[g_df['collision_state'] < 0]

In [ ]:
len(g_df_ems),len(g_df_abs)

In [ ]:
np.min(g_df['BE'])

In [ ]:
np.max(g_df['k_FD'])

In [ ]:
# def bosonic_processing(g_df,cart_qpts_df,enq_df,T):
#     """
#     This function takes a list of k-point indices and returns the Fermi-distributions and energies associated with each k-point on that list. The Fermi distributions are calculated with respect to a particular chemical potential.      
#     Parameters:
#     -----------
    
#     g_df : pandas dataframe containing:
    
#         k_inds : vector_like, shape (n,1)
#         Index of k point (pre-collision)
        
#         q_inds : vector_like, shape (n,1)
#         Index of q point
        
#         k+q_inds : vector_like, shape (n,1)
#         Index of k point (post-collision)
        
#         m_band : vector_like, shape (n,1)
#         Band index of post-collision state
        
#         n_band : vector_like, shape (n,1)
#         Band index of pre-collision state
        
#         im_mode : vector_like, shape (n,1)
#         Polarization of phonon mode
        
#         g_element : vector_like, shape (n,1)
#         E-ph matrix element
               
#     cart_qpts_df : pandas dataframe containing:
    
#         q_inds : vector_like, shape (n,1)
#         Index of q point
        
#         kx : vector_like, shape (n,1)
#         x-coordinate in Cartesian momentum space [1/m]    
        
#         ky : vector_like, shape (n,1)
#         y-coordinate in Cartesian momentum space [1/m]  
        
#         kz : vector_like, shape (n,1)
#         z-coordinate in Cartesian momentum space [1/m]
        
#     enq_df : pandas dataframe containing

#         k_inds : vector_like, shape (n,1)
#         Index of k point
        
#         im_mode : vector_like, shape (n,1)
#         Phonon polarization index
        
#         energy [Ryd] : vector_like, shape (n,1)
#         Energy associated with k point in Rydberg units
        
            
#     T : scalar
#     Lattice temperature in Kelvin
    
#     Returns:
#     --------
    
#     g_df : pandas dataframe containing:

#         k_inds : vector_like, shape (n,1)
#         Index of k point (pre-collision)
        
#         q_inds : vector_like, shape (n,1)
#         Index of q point
        
#         k+q_inds : vector_like, shape (n,1)
#         Index of k point (post-collision)
        
#         m_band : vector_like, shape (n,1)
#         Band index of post-collision state
        
#         n_band : vector_like, shape (n,1)
#         Band index of pre-collision state
        
#         im_mode : vector_like, shape (n,1)
#         Polarization of phonon mode
        
#         g_element : vector_like, shape (n,1)
#         E-ph matrix element
        
#         k_fermi_dist : vector_like, shape (n,1)
#         Fermi distribution of pre collision state
        
#         k+q_fermi_dist : vector_like, shape (n,1)
#         Fermi distribution of post collision state
        
#         k_energy : vector_like, shape (n,1)
#         Energy of the pre collision state
        
#         k+q_energy : vector_like, shape (n,1)
#         Energy of the post collision state
         
#     """
    
#     # Physical constants
#     e = 1.602*10**(-19) # fundamental electronic charge [C]
#     kb = 1.38064852*10**(-23); # Boltzmann constant in SI [m^2 kg s^-2 K^-1]
    
#     polar_vector = np.unique(g_df['im_mode'].values)
    
#     for i1 in range(len(polar_vector)):
#         polar_index = polar_vector[i1]
#         temp_g_inds = g_df['im_mode'].values == polar_index
#         temp_g_df = g_df.loc[temp_g_inds]
        
#         temp_enq_inds = enq_df['im_mode'].values == polar_index
#         temp_enq_df = enq_df.loc[temp_enq_inds]
        
#         index_vector = temp_g_df['q_inds'].values
#         for i2 in trange(len(index_vector)):
#             index = index_vector[i2]
            
#             index_slice = temp_enq_df['q_inds'].values == index
#             index_g_slice = temp_g_df['q_inds'].values == index
#             temp_g_df.loc[index_g_slice,'q_en [eV]'] = temp_enq_df.loc[index_slice,'energy [Ryd]'].values*13.6056980659
        
#         g_df.loc[temp_g_inds,'q_en [eV]'] = temp_g_df['q_en [eV]'].values
            
#     g_df = bose_distribution(g_df, T)
    
#     return g_df

In [ ]:
g_df = bosonic_processing(g_df,cart_qpts_df,enq_df,300)

In [ ]:
g_df

In [ ]:
g_df.loc[(g_df['collison_state'] == 0)]

In [ ]:
kb = 1.38064852*10**(-23); # Boltzmann constant in SI [m^2 kg s^-2 K^-1]
T = 300
e = 1.602*10**(-19)

In [ ]:
(np.exp(enq_df['energy [Ryd]'].values*13.6056980659*e/(kb*T))-1)**(-1)

In [ ]:
(np.exp((enk_df['energy [Ryd]'].values*13.6056980659 - 5.9)*e/(kb*T))+1)**(-1)

In [ ]:
len(np.unique(g_df['k_inds'])),len(np.unique(g_df['k+q_inds']))


In [ ]:
g_df.loc[np.multiply(g_df['k_inds'] == 1,g_df['k+q_inds'] == 271)]

## Data validation (Peishi Updated: 4/30)